# House Prices - Test Set Data Cleaning
### Biel Altimira

## 1. Carreguem les dades

In [4]:
import pandas as pd
import numpy as np

In [5]:
def load_data(file_name, path):
    full_path = os.path.join(path, file_name)
    return pd.read_csv(full_path)

In [6]:
#Carreguem a un dataframe les dades
file_name = "test.csv"
path = "data"
test = load_data(file_name, path)

In [7]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## 2. Neteja de dades

In [9]:
# Eliminem les columnes amb baixa correlació
to_drop = ["MoSold", "3SsnPorch", "BsmtFinSF2", "BsmtHalfBath", "MiscVal", "LowQualFinSF", "YrSold"]
test.drop(to_drop, axis=1, inplace=True)